In [7]:
import pandas as pd
import os
from openai import OpenAI

In [8]:
current_directory = os.getcwd()

parent_directory = os.path.dirname(current_directory)

DATA_PATH = os.path.join(parent_directory, "data")
DATA_TOURNAMENT = os.path.join(DATA_PATH, "tournament_hints_data.parquet")

DATA_TOURNAMENT = os.path.join(DATA_PATH, "tournament_hints_data.parquet")
DATA_BUSINESS = os.path.join(DATA_PATH, "Business category taxonomy.xlsx")
DATA_NAICS = os.path.join(
    DATA_PATH,
    "naics3_tournament_taxonomy.xlsx",
)

openai_api_key = "sk-proj-OSCk0tPAJxvpKtxmZIObT3BlbkFJA9hXFC8sq06DAxMmJgQl"

print("Current Directory:", current_directory)
print("Parent Directory:", parent_directory)

Current Directory: /Users/marianluca/Projects/HackingBigNumbers/Tournament/marian
Parent Directory: /Users/marianluca/Projects/HackingBigNumbers/Tournament


In [9]:
df_naics = pd.read_excel(DATA_NAICS)
df_business = pd.read_excel(DATA_BUSINESS)

In [10]:
client = OpenAI(api_key=openai_api_key)

In [27]:
from tqdm import tqdm

In [34]:
def find_naics_category(description, naics_df):
    responses = []

    # print(f"Try to function {description} with {label}")

    for label, label_description in tqdm(zip(
        naics_df["naics_label"], naics_df["description"]),total=len(naics_df)):
        prompt = f"Given` the description: '{description}',give the probability which match the category: '{label}' which is described as: '{label_description}'. Compute and return a probability score from 0 to 1. Return just the probability score."

        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "user", "content": prompt},
                ],
                max_tokens=90,
                temperature=0,
            )

            responses.append((label, response.choices[0].message.content.strip()))

        except Exception as e:
            print(f"An error occurred: {e}")

    result = sorted(responses, key=lambda x: x[1], reverse=True)[0][0]

    print(f"{result} ----> {description}")

    return result


# Mapping each business label to a NAICS category
df_business["Mapped NAICS Label"] = df_business["description"].apply(
    lambda desc: find_naics_category(desc, df_naics)
)

# Saving the mapped labels
df_business.to_csv("mapped_business_labels.csv", index=False)

 69%|██████▉   | 66/96 [01:02<00:32,  1.08s/it]

In [ ]:
# Mapping each business label to a NAICS category
df_business["Mapped NAICS Label"] = df_business["description"].apply(
    lambda desc: find_naics_category(desc, df_naics)
)

# Saving the mapped labels
df_business.to_csv("mapped_business_labels.csv", index=False)

In [ ]:
def find_naics_category_second_approach(labels, naics_df):
    responses = []

    # print(f"Try to function {description} with {label}")

    for label, label_description in zip(
        naics_df["naics_label"], naics_df["description"]
    ):
        prompt = f"Given` the description: '{label_description}' with  '{label}',choose one of the following labels: '{list(naics_df["nics"])}' which is described as: '{label_description}'. Compute and return a probability score from 0 to 1. Return just the probability score."

        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    # {
                    #     "role": "system",
                    #     "content": "You are tasked with evaluating business descriptions and categorizing them based on a likelihood score. For each provided description and category, compute and return a probability score from 0 to 1.",
                    # },
                    {"role": "user", "content": prompt},
                ],
                max_tokens=90,
                temperature=0,
            )

            responses.append((label, response.choices[0].message.content.strip()))

        except Exception as e:
            print(f"An error occurred: {e}")

    print(responses)

    result = sorted(responses, key=lambda x: x[1], reverse=True)[0][0]

    return result


# Mapping each business label to a NAICS category
df_business["Mapped NAICS Label"] = df_business["description"].apply(
    lambda desc: find_naics_category(desc, df_naics)
)

# Saving the mapped labels
df_business.to_csv("mapped_business_labels.csv", index=False)

In [23]:
test = df_business[:100]

In [24]:
test

,label,description
0,ATVs Dealers & Services,Businesses categorized under ATVs Dealers & Se...
1,Abortion Clinics,Abortion Clinics provide medical services rela...
2,Accounting & Bookkeeping Services,Accounting & Bookkeeping Services encompass a ...
3,Acupuncture clinic,An Acupuncture clinic is a healthcare facility...
4,Adhesives & Sealants,Adhesives & Sealants encompass a diverse range...
...,...,...
95,Butcher shop,A butcher shop is a retail establishment speci...
96,Buyback Shops,Buyback Shops are retail establishments that s...
97,CBD and Medical Marijuana,The CBD and Medical Marijuana category encompa...
98,Cabinet Makers,Cabinet Makers specialize in the skilled craft...
